<center>
<h2>Capstone Project:</h2> 
<h1>Identifying Locations for John's Ice Cream and Pizza Shop</h1>
</center>
<p></p>

<b>First import all necessary libraries</b>

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: ...working... 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/win-64::anaconda==2019.07=py37_0
  - defaults/win-64::numba==0.44.1=py37hf9181ef_0
done

## Package Plan ##

  environment location: C:\Users\rjia\AppData\Local\Continuum\anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _anaconda_depends-2019.03  |           py37_0           6 KB
    anaconda-custom            |           py37_1           3 KB
    ca-certificates-2019.9.11  |       hecc5488_0         181 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    openssl-1.1.1c             |       hfa6e2cd_0         4.7 MB  conda-forge
    tbb-2019.8  

<b>Read the CSV file containing a list of towns in McLean County along with their populations and locations into a dataframe</b>

In [87]:
df = pd.read_csv('towns_in_mclean_county_il.csv')
df.shape

(21, 4)

In [88]:
df.head()

,Town,Population,Latitude,Longitude
0,Anchor,142,40.5681,-88.5356
1,Arrowsmith,267,40.4495,-88.6317
2,Bellflower,416,40.3400,-88.5270
3,Bloomington,78368,40.4842,-88.9937
4,Carlock,583,40.5823,-89.1320


<b>Use geopy library to get the latitude and longitude values of McLean County</b>

To define an instance of the geocoder, a user_agent, named "mclean_explorer", is defined.

In [89]:
address = 'McLean County, IL'

geolocator = Nominatim(user_agent="mclean_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of McLean County, Illinois are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of McLean County, Illinois are 40.4631789, -88.8196613.


<b>Create a McLean County map with its all towns superimposed on it</b>

In [90]:
# create map of McLean County using latitude and longitude values

map_mclean = folium.Map(location=[latitude, longitude], zoom_start=9)

# add markers to map
for lat, lng, town in zip(df['Latitude'], df['Longitude'], df['Town']):
    label = '{}, IL'.format(town)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_mclean)  
    
map_mclean

<b>Foursquare credentials</b>

In [91]:
CLIENT_ID = 'OTVHEHACLVFKWMQAAEARJ2PGX1SZAWEI2Y1V5Y4JQPUEZZZ3' # Foursquare ID
CLIENT_SECRET = 'QQIEDAOXB4VDE5DMGFQ0ZGEZ3PJUQ0S2UIEIZZGRNNNE3BUQ' # Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OTVHEHACLVFKWMQAAEARJ2PGX1SZAWEI2Y1V5Y4JQPUEZZZ3
CLIENT_SECRET:QQIEDAOXB4VDE5DMGFQ0ZGEZ3PJUQ0S2UIEIZZGRNNNE3BUQ


<b>Display the first town in the dataframe</b>

In [92]:
df.loc[0, 'Town']

'Anchor'

<b>Get the town's latitude and longitude values</b>

In [93]:
town_latitude = df.loc[0, 'Latitude'] # town latitude value
town_longitude = df.loc[0, 'Longitude'] # town longitude value

town_name = df.loc[0, 'Town'] # town name

print('Latitude and longitude values of {} are {}, {}.'.format(town_name, 
                                                               town_latitude, 
                                                               town_longitude))

Latitude and longitude values of Anchor are 40.5681, -88.5356.


<b>Now get the top 100 venues in Anchor within a radius of 500 meters.

First create the GET request URL. </b>

In [94]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    town_latitude, 
    town_longitude, 
    radius, 
    LIMIT)

url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=OTVHEHACLVFKWMQAAEARJ2PGX1SZAWEI2Y1V5Y4JQPUEZZZ3&client_secret=QQIEDAOXB4VDE5DMGFQ0ZGEZ3PJUQ0S2UIEIZZGRNNNE3BUQ&v=20180605&ll=40.5681,-88.5356&radius=500&limit=100'

<b>Send the GET request and examine the resutls</b>

In [95]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d842821b9961d00359995b4'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Anchor',
  'headerFullLocation': 'Anchor',
  'headerLocationGranularity': 'city',
  'totalResults': 1,
  'suggestedBounds': {'ne': {'lat': 40.5726000045, 'lng': -88.52968714545935},
   'sw': {'lat': 40.5635999955, 'lng': -88.54151285454066}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4f32767819836c91c7da36ef',
       'name': 'Anchor Inn',
       'location': {'address': '248 2nd St',
        'lat': 40.568146,
        'lng': -88.536277,
        'labeledLatLngs': [{'label': 'display',
          'lat': 40.568146,
          'lng': -88.

<b>All information is in the <i>items</i> key. We next borrow the get_category_type function from the Foursquare lab.</b>

In [96]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

<b>Now we are ready to clean the json and structure it into a pandas dataframe.</b>

In [97]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Anchor Inn,Food,40.568146,-88.536277


<h2>Explore Towns in McLean County</h2>

Create a function to repeat the same process to all towns in McLean County

In [98]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Town', 
                  'Town Latitude', 
                  'Town Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

<b>Now run the above function on each neighborhood and create a new dataframe called <i>mclean_venues</i>.</b>

In [99]:
mclean_venues = getNearbyVenues(names=df['Town'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Anchor
Arrowsmith
Bellflower
Bloomington
Carlock
Chenoa
Colfax
Cooksville
Danvers
Downs
Ellsworth
Gridley
Heyworth
Hudson
Le Roy
Lexington
McLean
Normal
Saybrook
Stanford
Towanda


<b>Check the size of the resulting dataframe</b>

In [100]:
print(mclean_venues.shape)
mclean_venues.head()

(113, 7)


,Town,Town Latitude,Town Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Anchor,40.5681,-88.5356,Anchor Inn,40.568146,-88.536277,Food
1,Bellflower,40.3400,-88.5270,Addiction Boutique,40.341004,-88.526754,Boutique
2,Bellflower,40.3400,-88.5270,Don Harden Ball Field,40.340889,-88.529912,Baseball Field
3,Bloomington,40.4842,-88.9937,The Lucca Grill,40.482486,-88.992946,Italian Restaurant
4,Bloomington,40.4842,-88.9937,Bloomington Center For The Performing Arts,40.484045,-88.992847,Concert Hall


Let's check how many venues were returned for each neighborhood

In [101]:
mclean_venues.groupby('Town').count()

,Town Latitude,Town Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Town,,,,,,
Anchor,1,1,1,1,1,1
Bellflower,2,2,2,2,2,2
Bloomington,26,26,26,26,26,26
Carlock,4,4,4,4,4,4
Chenoa,6,6,6,6,6,6
Colfax,7,7,7,7,7,7
Cooksville,2,2,2,2,2,2
Danvers,4,4,4,4,4,4
Downs,4,4,4,4,4,4


<b>Check how many unique categories can be curated from all the returned venues</b>

In [102]:
print('There are {} uniques categories.'.format(len(mclean_venues['Venue Category'].unique())))

There are 68 uniques categories.


<h2>Analyze Each Town in McLean County</h2>

In [103]:
# one hot encoding
mclean_onehot = pd.get_dummies(mclean_venues[['Venue Category']], prefix="", prefix_sep="")

# add town column back to dataframe
mclean_onehot['Town'] = mclean_venues['Town'] 

# move town column to the first column
fixed_columns = [mclean_onehot.columns[-1]] + list(mclean_onehot.columns[:-1])
mclean_onehot = mclean_onehot[fixed_columns]

mclean_onehot

,Town,American Restaurant,Arcade,Athletics & Sports,Auto Garage,Bagel Shop,Bank,Bar,Baseball Field,Boutique,Breakfast Spot,Burger Joint,Café,Chinese Restaurant,Coffee Shop,College Library,Concert Hall,Convenience Store,Deli / Bodega,Diner,Dive Bar,Doctor's Office,Electronics Store,Farmers Market,Fast Food Restaurant,Flea Market,Food,Food Court,Football Stadium,Gas Station,Gay Bar,Grocery Store,Gym,Hardware Store,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Home Service,Hostel,Ice Cream Shop,Iraqi Restaurant,Italian Restaurant,Japanese Restaurant,Lake,Lawyer,Library,Lounge,Mexican Restaurant,Motel,Neighborhood,Nightlife Spot,North Indian Restaurant,Park,Pharmacy,Pizza Place,Playground,Plaza,Pool,Pub,Restaurant,Rock Club,Salon / Barbershop,Sandwich Place,Shop & Service,Sporting Goods Shop,Tapas Restaurant,Theme Park,Video Store
0,Anchor,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Bellflower,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Bellflower,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Bloomington,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Bloomington,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,Bloomington,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,Bloomington,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
7,Bloomington,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,Bloomington,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,Bloomington,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


<b>Find the size of the new dataframe</b>

In [104]:
mclean_onehot.shape

(113, 69)

<b>To group rows by town and by taking the mean of the frequency of occurrence of each category</b>

In [105]:
mclean_grouped = mclean_onehot.groupby('Town').mean().reset_index()
mclean_grouped

,Town,American Restaurant,Arcade,Athletics & Sports,Auto Garage,Bagel Shop,Bank,Bar,Baseball Field,Boutique,Breakfast Spot,Burger Joint,Café,Chinese Restaurant,Coffee Shop,College Library,Concert Hall,Convenience Store,Deli / Bodega,Diner,Dive Bar,Doctor's Office,Electronics Store,Farmers Market,Fast Food Restaurant,Flea Market,Food,Food Court,Football Stadium,Gas Station,Gay Bar,Grocery Store,Gym,Hardware Store,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Home Service,Hostel,Ice Cream Shop,Iraqi Restaurant,Italian Restaurant,Japanese Restaurant,Lake,Lawyer,Library,Lounge,Mexican Restaurant,Motel,Neighborhood,Nightlife Spot,North Indian Restaurant,Park,Pharmacy,Pizza Place,Playground,Plaza,Pool,Pub,Restaurant,Rock Club,Salon / Barbershop,Sandwich Place,Shop & Service,Sporting Goods Shop,Tapas Restaurant,Theme Park,Video Store
0,Anchor,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.000,0.000000,0.000000,0.000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.00,1.0,0.000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.0,0.00,0.0,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.000000,0.000,0.000
1,Bellflower,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.5,0.5,0.00,0.000000,0.000,0.000000,0.000000,0.000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.0,0.000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.0,0.00,0.0,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.000000,0.000,0.000
2,Bloomington,0.000000,0.000000,0.000000,0.000000,0.000,0.038462,0.192308,0.0,0.0,0.00,0.000000,0.000,0.038462,0.038462,0.000,0.038462,0.000000,0.000,0.000000,0.000000,0.038462,0.000,0.038462,0.000000,0.00,0.0,0.000,0.00,0.038462,0.038462,0.000000,0.00,0.00,0.00,0.00,0.038462,0.000000,0.00,0.000000,0.038462,0.0,0.038462,0.000,0.00,0.000000,0.00,0.00,0.038462,0.000000,0.0,0.00,0.0,0.000000,0.038462,0.000000,0.00,0.00,0.000000,0.038462,0.038462,0.038462,0.038462,0.115385,0.000,0.00,0.038462,0.000,0.000
3,Carlock,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.0,0.0,0.25,0.000000,0.000,0.000000,0.000000,0.000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.0,0.000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.0,0.00,0.0,0.500000,0.000000,0.000000,0.25,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.000000,0.000,0.000
4,Chenoa,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.166667,0.0,0.0,0.00,0.000000,0.000,0.000000,0.000000,0.000,0.000000,0.000000,0.000,0.000000,0.166667,0.000000,0.000,0.000000,0.000000,0.00,0.0,0.000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000,0.00,0.166667,0.00,0.00,0.000000,0.000000,0.0,0.00,0.0,0.166667,0.166667,0.166667,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.000000,0.000,0.000
5,Colfax,0.000000,0.000000,0.142857,0.000000,0.000,0.000000,0.142857,0.0,0.0,0.00,0.000000,0.000,0.000000,0.000000,0.000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.0,0.000,0.00,0.000000,0.000000,0.142857,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.142857,0.000000,0.0,0.000000,0.000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.0,0.00,0.0,0.142857,0.000000,0.142857,0.00,0.00,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.000000,0.000,0.000
6,Cooksville,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.0,0.0,0.00,0.500000,0.000,0.000000,0.000000,0.000,0.

<h4>Print each town along with the top 5 most common venues</h4>

In [106]:
num_top_venues = 5

for hood in mclean_grouped['Town']:
    print("----"+hood+"----")
    temp = mclean_grouped[mclean_grouped['Town'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Anchor----
                 venue  freq
0                 Food   1.0
1  American Restaurant   0.0
2         Neighborhood   0.0
3                Motel   0.0
4   Mexican Restaurant   0.0


----Bellflower----
                 venue  freq
0       Baseball Field   0.5
1             Boutique   0.5
2  American Restaurant   0.0
3                 Lake   0.0
4         Neighborhood   0.0


----Bloomington----
                venue  freq
0                 Bar  0.19
1      Sandwich Place  0.12
2         Coffee Shop  0.04
3  Chinese Restaurant  0.04
4             Gay Bar  0.04


----Carlock----
            venue  freq
0            Park  0.50
1      Playground  0.25
2  Breakfast Spot  0.25
3          Hostel  0.00
4  Ice Cream Shop  0.00


----Chenoa----
         venue  freq
0         Park  0.17
1     Pharmacy  0.17
2  Pizza Place  0.17
3          Bar  0.17
4     Dive Bar  0.17


----Colfax----
                venue  freq
0  Athletics & Sports  0.14
1              Hostel  0.14
2                Par

<b>Put that into a pandas dataframe</b>

First write a function to sort the venues in descending order.

In [107]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

<b>Now create the new dataframe to display the top 10 venues for each neighborhood.</b>

In [108]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Town']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
mclean_venues_sorted = pd.DataFrame(columns=columns)
mclean_venues_sorted['Town'] = mclean_grouped['Town']

for ind in np.arange(mclean_grouped.shape[0]):
    mclean_venues_sorted.iloc[ind, 1:] = return_most_common_venues(mclean_grouped.iloc[ind, :], num_top_venues)

mclean_venues_sorted

,Town,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Anchor,Food,Flea Market,Diner,Dive Bar,Doctor's Office,Electronics Store,Farmers Market,Fast Food Restaurant,Video Store,Theme Park
1,Bellflower,Boutique,Baseball Field,Food,Dive Bar,Doctor's Office,Electronics Store,Farmers Market,Fast Food Restaurant,Flea Market,Video Store
2,Bloomington,Bar,Sandwich Place,Mexican Restaurant,Gay Bar,Farmers Market,History Museum,Doctor's Office,Ice Cream Shop,Italian Restaurant,Concert Hall
3,Carlock,Park,Breakfast Spot,Playground,Video Store,Diner,Dive Bar,Doctor's Office,Electronics Store,Farmers Market,Fast Food Restaurant
4,Chenoa,Lawyer,Dive Bar,Park,Pharmacy,Bar,Pizza Place,Video Store,Electronics Store,Diner,Doctor's Office
5,Colfax,Grocery Store,Bar,Park,Pizza Place,Hostel,Pool,Athletics & Sports,Food Court,Football Stadium,Food
6,Cooksville,Burger Joint,Iraqi Restaurant,Video Store,Flea Market,Dive Bar,Doctor's Office,Electronics Store,Farmers Market,Fast Food Restaurant,Food
7,Danvers,Concert Hall,Home Service,Bar,Plaza,Flea Market,Dive Bar,Doctor's Office,Electronics Store,Farmers Market,Fast Food Restaurant
8,Downs,Home Service,Football Stadium,Ice Cream Shop,Video Store,Flea Market,Dive Bar,Doctor's Office,Electronics Store,Farmers Market,Fast Food Restaurant
9,Ellsworth,Park,Video Store,Fast Food Restaurant,Diner,Dive Bar,Doctor's Office,Electronics Store,Farmers Market,Flea Market,Convenience Store


In [109]:
mclean_venues

,Town,Town Latitude,Town Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Anchor,40.5681,-88.5356,Anchor Inn,40.568146,-88.536277,Food
1,Bellflower,40.3400,-88.5270,Addiction Boutique,40.341004,-88.526754,Boutique
2,Bellflower,40.3400,-88.5270,Don Harden Ball Field,40.340889,-88.529912,Baseball Field
3,Bloomington,40.4842,-88.9937,The Lucca Grill,40.482486,-88.992946,Italian Restaurant
4,Bloomington,40.4842,-88.9937,Bloomington Center For The Performing Arts,40.484045,-88.992847,Concert Hall
5,Bloomington,40.4842,-88.9937,Coffee Hound,40.481628,-88.993712,Coffee Shop
6,Bloomington,40.4842,-88.9937,Jimmy John's,40.482683,-88.995403,Sandwich Place
7,Bloomington,40.4842,-88.9937,Fat Jack's,40.482735,-88.993817,Bar
8,Bloomington,40.4842,-88.9937,Taqueria El Porton,40.486955,-88.993647,Mexican Restaurant
9,Bloomington,40.4842,-88.9937,Fox & Hounds Hair Studio & Day Spa,40.481022,-88.994887,Salon / Barbershop


In [110]:
mclean_grouped2 = mclean_grouped[['Town','Ice Cream Shop','Pizza Place']]
mclean_grouped2

,Town,Ice Cream Shop,Pizza Place
0,Anchor,0.000000,0.000000
1,Bellflower,0.000000,0.000000
2,Bloomington,0.038462,0.000000
3,Carlock,0.000000,0.000000
4,Chenoa,0.000000,0.166667
5,Colfax,0.000000,0.142857
6,Cooksville,0.000000,0.000000
7,Danvers,0.000000,0.000000
8,Downs,0.250000,0.000000
9,Ellsworth,0.000000,0.000000


<b>To merge the dataframe with restauraunt information and the one with population numbers</b>

In [111]:
mclean_grouped3 = pd.merge(mclean_grouped2,df, on='Town')

<b>To remove all towns that have an ice cream shop</b>

In [112]:
df_no_ice = mclean_grouped3[mclean_grouped3['Ice Cream Shop'] == 0]
df_no_ice

,Town,Ice Cream Shop,Pizza Place,Population,Latitude,Longitude
0,Anchor,0.0,0.000000,142,40.5681,-88.5356
1,Bellflower,0.0,0.000000,416,40.3400,-88.5270
3,Carlock,0.0,0.000000,583,40.5823,-89.1320
4,Chenoa,0.0,0.166667,1840,40.7417,-88.7198
5,Colfax,0.0,0.142857,1081,40.5670,-88.6164
6,Cooksville,0.0,0.000000,170,40.5431,-88.7165
7,Danvers,0.0,0.000000,1291,40.5295,-89.1773
9,Ellsworth,0.0,0.000000,276,40.4503,-88.7167
10,Gridley,0.0,0.000000,1426,40.7434,-88.8815
11,Heyworth,0.0,0.000000,3031,40.3134,-88.9737


<b>Then remove all towns that have a pizza place</b>

In [119]:
df_final = df_no_ice[df_no_ice['Pizza Place'] == 0]
df_final

,Town,Ice Cream Shop,Pizza Place,Population,Latitude,Longitude
0,Anchor,0.0,0.0,142,40.5681,-88.5356
1,Bellflower,0.0,0.0,416,40.3400,-88.5270
3,Carlock,0.0,0.0,583,40.5823,-89.1320
6,Cooksville,0.0,0.0,170,40.5431,-88.7165
7,Danvers,0.0,0.0,1291,40.5295,-89.1773
9,Ellsworth,0.0,0.0,276,40.4503,-88.7167
10,Gridley,0.0,0.0,1426,40.7434,-88.8815
11,Heyworth,0.0,0.0,3031,40.3134,-88.9737
13,Le Roy,0.0,0.0,3570,40.3520,-88.7642
15,McLean,0.0,0.0,779,40.3145,-89.1698


<b>Sort towns by population in the decending order, and then display the top 3 towns</b>

In [129]:
df_final3 = df_final.sort_values(by='Population', ascending=False).head(3)
df_final3

,Town,Ice Cream Shop,Pizza Place,Population,Latitude,Longitude
16,Normal,0.0,0.0,54534,40.5142,-88.9906
13,Le Roy,0.0,0.0,3570,40.3520,-88.7642
11,Heyworth,0.0,0.0,3031,40.3134,-88.9737


<b>Display the McLean County map with the three towns superimposed on it </b>

In [131]:
# create map of McLean County using latitude and longitude values

map_mclean = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, town in zip(df_final3['Latitude'], df_final3['Longitude'], df_final3['Town']):
    label = '{}, IL'.format(town)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_mclean)  
    
map_mclean

<b>Conclusions:  
    
    Three towns that meet the selection criteria have been identified: Normal, Le Roy, and Heyworth.
    Among them, Normal has a much larger population and is likely the best location for the proposed shop.
    
</b> 